In [2]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 103.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 116.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 22.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from datasets import load_dataset, load_metric
from transformers import DistilBertTokenizerFast
from transformers import AutoModelForSequenceClassification,DataCollatorWithPadding
from transformers import Trainer, TrainingArguments

In [4]:
model_name = 'distilbert-base-uncased'

In [5]:
dataset = load_dataset("boolq")

Generating train split:   0%|          | 0/9427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3270 [00:00<?, ? examples/s]

Dataset boolq downloaded and prepared to /root/.cache/huggingface/datasets/boolq/default/0.1.0/bf0dd57da941c50de94ae3ce3cef7fea48c08f337a4b7aac484e9dddc5aa24e5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'answer', 'passage'],
        num_rows: 3270
    })
})


In [8]:
dataset['train'][0]

{'question': 'do iran and afghanistan speak the same language',
 'answer': True,
 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}

To begin data processing, we need to create a text tokenizer. Transformers come with a bunch of already pretrained tokenizers. So, we will use one of them

In [9]:
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

now we will define function that tokenize and encode input with tokenizer as well as prepare labels field.

In [12]:
def tokenize_data(example):
  encoded = tokenizer(example['question'],example['passage'],truncation=True)
  encoded['labels'] = [int(a) for a in example['answer']]

  return encoded

tokenized_datasets = dataset.map(tokenize_data,batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

now, we need to define a data collator, which will create batches of examples that don’t have the same length.

In [17]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Download pretrained Model

In [18]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

Now, Define training Arguments

In [19]:
args = TrainingArguments('roberta-booql', per_device_train_batch_size=16, learning_rate=1e-3, num_train_epochs=20)

Putting all objects defined above, together into an instance of trainer class

In [20]:
trainer = Trainer(model, args, train_dataset=tokenized_datasets['train'],
                  eval_dataset = tokenized_datasets['validation'],
                  data_collator = data_collator,
                  tokenizer = tokenizer)

Now, Finetuning the model

In [21]:
#https://asperbrothers.com/blog/question-answering-python/
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: question, answer, passage. If question, answer, passage are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 9427
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11800
  Number of trainable parameters = 66955010
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokeni

Step,Training Loss
500,0.671900
1000,0.672200
1500,0.663200
2000,0.663700
2500,0.668900
3000,0.661900
3500,0.663700
4000,0.664000
4500,0.665300
5000,0.665600


Saving model checkpoint to roberta-booql/checkpoint-500
Configuration saved in roberta-booql/checkpoint-500/config.json
Model weights saved in roberta-booql/checkpoint-500/pytorch_model.bin
tokenizer config file saved in roberta-booql/checkpoint-500/tokenizer_config.json
Special tokens file saved in roberta-booql/checkpoint-500/special_tokens_map.json
Saving model checkpoint to roberta-booql/checkpoint-1000
Configuration saved in roberta-booql/checkpoint-1000/config.json
Model weights saved in roberta-booql/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in roberta-booql/checkpoint-1000/tokenizer_config.json
Special tokens file saved in roberta-booql/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to roberta-booql/checkpoint-1500
Configuration saved in roberta-booql/checkpoint-1500/config.json
Model weights saved in roberta-booql/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in roberta-booql/checkpoint-1500/tokenizer_config.json
Special tok

TrainOutput(global_step=11800, training_loss=0.6642070304741293, metrics={'train_runtime': 5055.9497, 'train_samples_per_second': 37.291, 'train_steps_per_second': 2.334, 'total_flos': 1.386027348894156e+16, 'train_loss': 0.6642070304741293, 'epoch': 20.0})

In [22]:
#Some Links to understand
#attention mask----https://huggingface.co/docs/transformers/glossary#attention-mask
#preprocessing----https://huggingface.co/docs/transformers/preprocessing

Generate predictions

In [26]:
predictions = trainer.predict(tokenized_datasets["validation"])
y_pred = predictions.predictions.argmax(-1)
labels = predictions.label_ids

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: question, answer, passage. If question, answer, passage are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 3270
  Batch size = 8


In [27]:
metric = load_metric("accuracy")

In [28]:
metric.compute(predictions=y_pred, references=predictions.label_ids)

{'accuracy': 0.6217125382262997}

62.17% accuracy is not bad for this small dataset